# Preprocessing

In [1]:
import glob
import pickle
import pandas as pd
import re
import dateparser
import numpy as np

In [ ]:
dataloc = "/home/hennes/thesis/Data/"
df = pd.read_csv(dataloc+"dataset.csv")

### Some preliminary stuff

# unify outlet names
df.loc[df["page"].str.startswith('Canal13'), "page"] = "Canal13"
df.loc[df["page"].str.startswith('100% Noticias'), "page"] = "100% Noticias"
df.loc[df["page"].str.startswith('Confidencial'), "page"] = "Confidencial"
df.loc[df["page"].str.startswith('Radio Corporacion'), "page"] = "Radio Corporacion"
df.loc[df["page"].str.startswith('Canal10'), "page"] = "Canal10"
df.loc[df["page"].str.startswith('Canal14'), "page"] = "Canal14"



# extract canal13 dates 
df.loc[df["page"] == 'Canal13', ["date"]] = df.loc[df["page"] == 'Canal13',
                                                 "url"].str.extract(r'(\d\d\d\d/\d\d/\d\d)').values
# extract canal 6 dates
df.loc[df["page"] == 'Canal6', ["date"]] = df.loc[df["page"] == 'Canal6',

                                                "url"].str.extract(r'(\d\d\d\d/\d\d/\d\d)').values
# extract radio 800 dates
df.loc[df["page"] == 'Radio 800', ["date"]] = df.loc[df["page"] == 'Radio 800',
                                                   "url"].str.extract(r'(\d\d\d\d/\d\d/\d\d)').values

# convert canal10 date to datetime
df10 = df.loc[df["page"] == "Canal10"]
df10.loc[:,'date'] = df10['date'].str.replace('de ', '', regex=True)
df10.loc[:,'date'] = df10['date'].str.replace(r'Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday', '', regex=True)
df10.loc[:,'date'] = pd.to_datetime(df10['date'])
df.loc[df["page"] == "Canal10", 'date'] = df10["date"].to_list()

# convert canal14 date to datetime
# first delete entries where only time of day was extracted
df.loc[df["page"] == "Canal14"] = df.loc[(df["page"] == "Canal14") & (df["date"].str.contains(r"\d{4}", na=False))]
# I am doing that with dateparser, because pandas cannot handle the spanish dates
df14 = df.loc[df["page"] == "Canal14"]
df14.loc[:,'date'] = df14.loc[:,'date'].apply(lambda x: dateparser.parse(x, settings={'STRICT_PARSING': True}))
df.loc[df["page"] == "Canal14", 'date'] = df14["date"].to_list()

# 100 % noticias
df100 = df.loc[df["page"] == "100% Noticias"]
# convert nan in date column to empty string so that dateparser works
df100.fillna({"date":" "}, axis=0, inplace=True)
df100.loc[:,'date'] = df100.loc[:,'date'].apply(lambda x: dateparser.parse(x, settings={'STRICT_PARSING': True}))
df.loc[df["page"] == "100% Noticias", 'date'] = df100["date"].to_list()

# canal2, canal4, confidencial, radio corporacion, radio nicaragua,
# radio primerissima are already in datetime format

# convert the rest to datetime
df.loc[:,"date"] = pd.to_datetime(df["date"])
# create year variable
df["year"] = pd.DatetimeIndex(df.date).year

df.to_csv("dataset1.csv", index = False)

<ipython-input-7-46384b245f3e>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df10.loc[:,'date'] = df10['date'].str.replace('de ', '', regex=True)
<ipython-input-7-46384b245f3e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df10.loc[:,'date'] = df10['date'].str.replace(r'Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday', '', regex=True)
<ipython-input-7-46384b245f3e>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

## Cleaning Data

### Pre-Tokenization Cleanup

In [5]:
# load dataset
df = pd.read_csv("dataset1.csv")

# delete duplicate articles
df.drop_duplicates(subset="title", inplace=True, ignore_index=True)

# check for articles that do not have any text in them
print("Number of articles without text per outlet")
print(df[df.text.isna()].groupby('page').size())

# there are quite some articles without text. I randomly selected some of these to verify that
# this is not a mistake in my crawler, but that they just don't have text. Indeed, they are all
# articles that only contain videos. This is reflected by the fact that the outlets in thise list
# are mostly TV and radio stations.

print("\nNumber of articles without text per year")
print(df[df.text.isna()].groupby('year').size())

# Because the models need text to work, I will discard rows without text

df = df.loc[~df.text.isna()].reset_index(drop = True)

# there is also a discrepancy in when the outlets started their work, as can be seen from the following table
print(df.groupby(['year', "page"]).agg({"text":np.size}).reset_index().pivot("page", "year", "text"))

# Because before 2015, only a single government outlet uploaded articles, I will keep only articles published
# from 2015 on

df = df.loc[df["year"]>2014]

# removing boilerplate, news agency sources, hyperlinks etc.

             # newlines, tabs etc.
repl_dict = {r"\t|\n|\r|\xa0":" ",
             # whitespace
             r"\s{2,}":" "}
df.replace({"text":repl_dict, "title":repl_dict}, regex=True, inplace = True)

            # hashtags
repl_dict = {"#": "",
             # source in some articles
            r'Fuente: El 19 Digital|Fuente: TN8':"",
             # all hyperlinks
            r"http\S+":"",
             # canal10 boilerplate
            r"Foto: Shutterstock.*": "",
            r"Foto:.*{.*": "",
            "Noticias de Nicaragua y el Mundo": "",
            r"p(\.\w+\s|\s)\{.*?\}": "",
            r"\{.*?\}": "",
            r"Normal.*X-NONE( /\* Style Definitions \*/ table\.MsoNormalTable)?": "",
             # canal4 boilerplate
            r"Canal 4 Noticias[\s\S]+Canal 4 Nicaragua. Todos los derechos reservados": "",
            r"Comparte[.\s]*?esto:[.\s]*?Tweet[.\s]*?WhatsApp[.\s]*?Telegram": "",
            "LEER TAMBIÉN": "",
            "Leer más:": "",
            "AMPLIACIÓN EN BREVE…": "",
            r"Canal 4 Noticias de Nicaragua.*": "",
             "Periodista en Multinoticias, Canal 4": "",
             # news agency
            "(EFE)":"",
             # source mentioned at end of article
            r"Con información de\:+$": "",
             # copyright stuff
            "© 100% Noticias ¡Con primicias a toda hora!":"",
            "© Getty Images":"",
            r"©\s?[Vv]iva [Nn]icaragua,? (Canal 13 )?(Previous Next)?": "",
            "© AFP":"",
            "© AP":"",
            "© creative commons": "",
            "© El 19 Digital": "",
            "© Consejo de Comunicación y Ciudadanía": "",
            "© Juventud Presidente": "",
            "© Ministerio de Gobernación": "",
             # copyrights for photographers
            r"\w+?\s+?\w+?\s+?©":"",
             # article suggestions
            "Te recomendamos:": "",
            "Quizás te interesa:":"",
            "Lee Aquí:": "",
             # twitter links
            r"pic\.twitter\.com.+?\d{4}": "",
            r"—.+?\(@.+?\).+?\d{4}":"",
             # source information
            r"Con información de:.+$": "",
            # emails
            r"\[email protected\]":""}

df.replace({"text":repl_dict, "title":repl_dict}, regex=True, inplace = True)

# the same goes for articles with very short texts.
# again, due to the focus on videos, some articles in the dataset consist only
# of a single, short sentence. Articles with less than 100-character long texts will
# therefore likewise be deleted.

df = df.loc[df["text"].apply(lambda x: len(str(x)) > 200)].reset_index(drop = True)

df.to_csv("dataset_token_ready.csv", index = False)

# save text files separately as csv file
df["text"].to_csv("texts.csv", index = False)

Number of articles without text per outlet
page
100% Noticias                 29
Canal10                     1070
Canal13                     9935
Canal2.pkl                    31
Canal4.pkl                     2
Canal6.pkl                    10
Radio Corporacion            119
Radio Nicaragua.pkl          111
Radio la Primerisima.pkl       7
dtype: int64

Number of articles without text per year
year
2011.0      72
2012.0    1611
2013.0    3408
2014.0    1874
2015.0    2068
2016.0    1152
2017.0     270
2018.0     292
2019.0     108
2020.0     389
2021.0      51
2022.0       4
dtype: int64
year                      2011.0  2012.0  2013.0  2014.0  2015.0  2016.0  \
page                                                                       
100% Noticias                NaN     NaN     NaN     NaN  1742.0  3236.0   
Canal10                      NaN     NaN     NaN     NaN     NaN  3517.0   
Canal13                    167.0   550.0     6.0  1820.0  2162.0  4963.0   
Canal14               

## Tokenization

In [1]:
import pickle
import pandas as pd
import spacy
import gensim
import csv

# save only text as csv file
df = pd.read_csv("dataset_token_ready.csv")["text"]

df.to_csv("texts.csv", index = False)

# load text as generator
csv_gen = (row for row in open("texts.csv"))

# load text as list
textlist = pd.read_csv("dataset_token_ready.csv")["text"].tolist()

# load spanish language model
nlp = spacy.load('es_core_news_md', disable=["parser", "attribute_ruler", "ner"])
# add stopwords
nlp.Defaults.stop_words |= {"a","y", "o"}
# reload language model to incorporate new stopwords 
nlp = spacy.load('es_core_news_md', disable=["parser", "attribute_ruler", "ner"])

tokenlist = []
# go through rows of article texts
for i, doc in enumerate(nlp.pipe(csv_gen, disable=["parser", "attribute_ruler", "ner"], n_process=4)):
    # append lists of lemmatised tokens to tokenlist
    tokenlist.append([token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha])
    if i % 10000 == 0 and i != 0: print(i)
# remove first entry, which is the column title
tokenlist = tokenlist[1:]
# save to pickle
with open(f"tokens/tokens.pkl", "wb") as f:
    pickle.dump(tokenlist, f)
    
# integrate tokenlist into df, useful for later structural topic model and all subsampling
# load tokenlist
with open("tokens/tokens.pkl", "rb") as f:
    tokenlist = pickle.load(f)

# integrate tokenlist back into dataframe
df = pd.read_csv("dataset_token_ready.csv")
df = df.assign(tokens=tokenlist)

# save dataset as pickle (csv does not work, because it cannot save the lists of tokens as cell entries)
df.to_pickle("dataset_with_tokens.pkl")

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
# select tokens and subsample
tokenlist = pd.read_pickle("dataset_with_tokens.pkl")[["page", "tokens"]]
tokenlist = tokenlist["tokens"].tolist()

# make bigram and trigram mod
bigram = gensim.models.Phrases(tokenlist, min_count = 100, threshold = 100)
trigram = gensim.models.Phrases(bigram[tokenlist], threshold = 130)

trigram_mod = gensim.models.phrases.Phraser(trigram)  
bigram_mod = gensim.models.phrases.Phraser(bigram)

tokenlist = [trigram_mod[bigram_mod[text]] for text in tokenlist]

tokens_trigrams = []
for doc in tokenlist:
    tokenstring = [" ".join([token for token in doc])]
    tokens_trigrams.append(tokenstring)

with open("tokens/tokens_trigrams.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerows(tokens_trigrams)